<a href="https://colab.research.google.com/github/imshiv-10/ML-SkLearn/blob/main/3.%20Decision%20Trees%20and%20Hyperparameters/decision_trees_random_forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### practical and coding-focused approach. We'll learn how to use decision trees and random forests to solve a real-world problem from Kaggle:

As a data scientist at the Bureau of Meteorology, you are tasked with creating a fully-automated system that can use today's weather data for a given location to predict whether it will rain at the location tomorrow.

In [1]:
import opendataset as od

ModuleNotFoundError: ignored